In [1]:
from dariah_topics import preprocessing as pre
from dariah_topics import visualization as visual
from dariah_topics import mallet as mal
from dariah_topics import evaluation

c:\users\philip\appdata\local\programs\python\python35-32\lib\site-packages\gensim\utils.py:855: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
c:\users\philip\appdata\local\programs\python\python35-32\lib\site-packages\funcy\decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
c:\users\philip\appdata\local\programs\python\python35-32\lib\site-packages\funcy\decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
c:\users\philip\appdata\local\programs\python\python35-32\lib\site-packages\funcy\decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
c:\users\philip\appdata\local\programs\python\python35-32\lib\site-packages\funcy\decorators.py

## Preprocessing

#### Liste mit Dateinamen erzeugen

In [114]:
#path_txt = "corpus_txt"
path_txt = "grenzbote_plain/*/"
#path_txt = "wiki/"

doclist_txt = pre.create_document_list(path_txt)
assert doclist_txt, "No documents found"
doclist_txt[:5]

['grenzbote_plain\\Jg_26_1867\\1867_Abseit_der_Heerstraße_43.txt',
 'grenzbote_plain\\Jg_26_1867\\1867_Alt_Mecklenburg_auf_dem_Rückzuge_60.txt',
 'grenzbote_plain\\Jg_26_1867\\1867_Anfang_und_Ende_des_mexikanischen_Kaiserreichs_59.txt',
 'grenzbote_plain\\Jg_26_1867\\1867_Aus_dem_Musikleben_Wiens_32.txt',
 'grenzbote_plain\\Jg_26_1867\\1867_Aus_der_Provinz_Preußen_45.txt']

#####  Liste mit Dokumentenlabels erzeugen - (Funktion wird durch Thorsten's generischere Funktion ersetzt)

In [115]:
doc_labels = list(pre.get_labels(doclist_txt))
doc_labels[:5]

['grenzbote_plain\\Jg_26_1867\\1867_Abseit_der_Heerstraße_43.txt',
 'grenzbote_plain\\Jg_26_1867\\1867_Alt_Mecklenburg_auf_dem_Rückzuge_60.txt',
 'grenzbote_plain\\Jg_26_1867\\1867_Anfang_und_Ende_des_mexikanischen_Kaiserreichs_59.txt',
 'grenzbote_plain\\Jg_26_1867\\1867_Aus_dem_Musikleben_Wiens_32.txt',
 'grenzbote_plain\\Jg_26_1867\\1867_Aus_der_Provinz_Preußen_45.txt']

#### Corpus laden

In [116]:
corpus_txt = pre.read_from_txt(doclist_txt)

#### Tokenisieren

In [117]:
doc_tokens = [list(pre.tokenize(txt)) for txt in list(corpus_txt)]

#### Create Dictionaries

In [118]:
id_types, doc_ids = pre.create_dictionaries(doc_labels, doc_tokens)

#### Sparse BOW Model

In [119]:
sparse_bow = pre.create_mm(doc_labels, doc_tokens, id_types, doc_ids)

In [120]:
sparse_bow[:5]

0
doc_id token_id   
1      73728     1
       3         2
       13654     1
       32774     2
       49160     1

## Topic Modeling with Gensim

##### Saving Sparse BOW

In [121]:
pre.save_bow_mm(sparse_bow, "gensim_txt")

#### Import Market Matrix

In [122]:
from gensim.models import LdaModel
from gensim.corpora import MmCorpus

In [123]:
mm = MmCorpus("gensim_txt.mm")

In [124]:
doc2id = {value : key for key, value in doc_ids.items()}
type2id = {value : key for key, value in id_types.items()}

##### Remove Features

In [125]:
import os.path
basepath = os.path.abspath('.')

with open(os.path.join(basepath, "tutorial_supplementals", "stopwords", "de.txt"), 'r', encoding = 'utf-8') as f: 
    stopword_list = f.read().split('\n')
    
stopword_list = set(stopword_list)

In [126]:
hapax_from_remove = pre.find_hapax(sparse_bow, id_types)
stopwords_from_remove = set(pre.find_stopwords(sparse_bow, id_types))

features_to_be_removed = hapax_from_remove.union(set(stopwords_from_remove))

sparse_bow_short = pre.remove_features(sparse_bow, id_types,features_to_be_removed)

In [127]:
pre.save_bow_mm(sparse_bow_short, "gensim_txt")

In [128]:
mm = MmCorpus("gensim_txt.mm")

#### Convert sparse_bow to list of (doc, tokens) tuples (like doc2bow)

In [129]:
doc2bow_list = []

for doc in sparse_bow_short.index.groupby(sparse_bow_short.index.get_level_values('doc_id')):
    temp = [(token, count) for token, count in zip(sparse_bow_short.loc[doc].index, sparse_bow_short.loc[doc][0])]
    doc2bow_list.append(temp)

In [130]:
len(doc2bow_list)

363

In [131]:
#model = LdaModel(corpus=mm, id2word=type2id, num_topics=60, alpha = "symmetric", passes = 10) #import momentan in visual 
# -> da ich mir noch nicht sicher bin, welche Funktionen in das tm_gensim.py sollen
model = LdaModel(corpus=mm, id2word=type2id, num_topics=20, passes = 100, iterations = 20)

In [132]:
model.get_document_topics(doc2bow_list[1])

[(5, 0.25933222450305737),
 (8, 0.2112252069643332),
 (9, 0.20394410102188393),
 (12, 0.018489225374997861),
 (15, 0.12839681509675954),
 (16, 0.020397626024924599),
 (17, 0.076909746118828767),
 (19, 0.068261576296772444)]

In [133]:
#Anzeige der keywords für topic n
n = 1
topic_nr_x = model.get_topic_terms(n)

topicTerms = [type2id[i[0]] for i in topic_nr_x]
topicTerms

['talentvoll',
 'hält',
 'hoffnungsvollsten',
 'kaltbereitcnder',
 'emeuten',
 'lüntzcl',
 'caut',
 'fahrender',
 'heerescrgänzungsgesetz',
 'völkerrechtes']

In [134]:
#Topic probabilities
topics = model.show_topics(num_topics = 40)
topics

[(0,
  '0.006*"fürchterlichem" + 0.004*"großalmosenier" + 0.003*"liebkosen" + 0.002*"unnatürlicher" + 0.002*"allegorischen" + 0.002*"wiederspiegelt" + 0.002*"urgiren" + 0.002*"russisch-nationalen" + 0.002*"gräßlicherer" + 0.002*"phantast"'),
 (1,
  '0.005*"talentvoll" + 0.003*"hält" + 0.002*"hoffnungsvollsten" + 0.002*"kaltbereitcnder" + 0.002*"emeuten" + 0.002*"lüntzcl" + 0.002*"caut" + 0.002*"fahrender" + 0.002*"heerescrgänzungsgesetz" + 0.002*"völkerrechtes"'),
 (2,
  '0.006*"sisti" + 0.003*"bornemannschen" + 0.002*"besatzungsrechts" + 0.002*"vorpetrinischen" + 0.002*"schauen" + 0.002*"ädreßdebatte" + 0.002*"feldmarschall" + 0.001*"mai»in" + 0.001*"caja" + 0.001*"ankündigungsschrift"'),
 (3,
  '0.009*"beigetragen" + 0.003*"acta" + 0.002*"strecke" + 0.002*"wohlsein" + 0.002*"publ&gt" + 0.002*"brustton" + 0.001*"paulinen" + 0.001*"tonlos" + 0.001*"unzuträglichkeit" + 0.001*"ni»ige"'),
 (4,
  '0.006*"theodor" + 0.003*"zurückblickt" + 0.003*"zell" + 0.003*"gedankensystem" + 0.003*"ausar

In [135]:
import pandas as pd
import regex as re
def gensim2dataframe(model):
    num_topics = model.num_topics
    topics_df = pd.DataFrame(index = range(num_topics), columns= range(10))

    topics = model.show_topics(num_topics)
    
    for topic_dist in topics:    
        idx = topic_dist[0]
        temp = re.findall(r"[A-züäöß\-\\]+", topics[idx][1])
        topics_df.loc[idx] = temp
    
    return topics_df

In [136]:
topics_df = gensim2dataframe(model)

ValueError: cannot copy sequence with size 11 to array axis with dimension 10

In [ ]:
topics_df

## Visualisierung 

In [ ]:
'''lda_model = 'out_easy/corpus.lda'
corpus = 'out_easy/corpus.mm'
dictionary = 'out_easy/corpus.dict'
doc_labels = 'out_easy/corpus_doclabels.txt'
interactive  = False

vis = visual.Visualization(lda_model, corpus, dictionary, doc_labels, interactive)'''

##### Create a document-topic matrix (that is a pandas data frame actually)

In [ ]:
doc_topic = visual.create_doc_topic(mm, model, doc_labels)

##### Visualize document-topic matrix

In [ ]:
%matplotlib inline
visual.doc_topic_heatmap(doc_topic)

##### Visualize topic distribution in a single document

In [ ]:
visual.plot_doc_topics(doc_topic, 0)

![success](http://cdn2.hubspot.net/hub/128506/file-446943132-jpg/images/computer_woman_success.jpg)